In [3]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver

In [4]:
# 코드를 통한 웹 요청이 차단된 사이트
# url = "https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=283&sid1=105&date=20210129&page=1"
# resp = requests.get(url)

In [8]:
# selenium을 사용해서 웹 요청

path = "D:\ssac-one2\dev-tools\chromedriver.exe"
browser = webdriver.Chrome(executable_path=path)

In [9]:
browser.get("https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=283&sid1=105&date=20210129&page=1")

In [11]:
browser.page_source[:500]

'<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"><head>\n<meta charset="euc-kr">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="referrer" contents="always">\n<meta http-equiv="refresh" content="600">\n<meta name="viewport" content="width=1106">\n\n    \n    \n        \n            \n                \n                    \n                    \n                    \n                    \n     '

In [12]:
browser.close()